In [1]:
!pip install opencv-python

In [5]:
import cv2
import numpy as np
import os

In [6]:
cap = cv2.VideoCapture(0)

classNames = open('coco.names').read().strip().split('\n')
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classNames), 3), dtype='uint8')
# classNames = ['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
ModelConfiguration = 'yolov3.cfg'
ModelWeights = 'yolov3.weights'

net = cv2.dnn.readNetFromDarknet(ModelConfiguration, ModelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)


def detectObjects(outputs, img):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confidences = []
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > 0.5:
                w, h = int(det[2] * wT), int(det[3] * hT)
                x, y = int(det[0] * wT - w / 2), int(det[1] * hT - h / 2)
                bbox.append([x, y, w, h])
                classIds.append(classId)
                confidences.append(float(confidence))
    # print(bbox)
    indices = cv2.dnn.NMSBoxes(bbox, confidences, 0.5, 0.3)
    for i in indices:
        i = i
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        label = classNames[classIds[i]] + " " + str(round(confidences[i], 2))
        cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

while True:
    ret, frame = cap.read()
    blob = cv2.dnn.blobFromImage(frame, 1 / 255, (416, 416), [0, 0, 0], 1, crop=False)
    net.setInput(blob)
    
    layerNames = net.getLayerNames()
    # print(layerNames)
    outputNames = [layerNames[i - 1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(outputNames)

    detectObjects(outputs, frame)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        break


KeyboardInterrupt: 